# Kölner Phonetik #43
Issue link: https://github.com/CorrelAid/compgen-ii-cgv/issues/43

In [1]:
import pandas as pd
import numpy as np
from compgen2 import Gov, Matcher, const, Phonetic
from pathlib import Path
import sys

In [2]:
data_root = "../data"
gov = Gov(data_root)

In [3]:
gov.load_data()

In [4]:
gov.build_indices()

## Kölner Phonetik

In [5]:
ph = Phonetic()

In [6]:
from collections import defaultdict

In [7]:
phonetic_by_name = {}
names_by_phonetic = defaultdict(set)
for n in gov.ids_by_name.keys():
    code = ph.encode(n)
    phonetic_by_name[n] = code
    names_by_phonetic[code] |= {n}
names_by_phonetic.default_factory = None

In [8]:
len(gov.ids_by_name.keys()), len(set(phonetic_by_name.values()))

(209019, 80901)

## Beispiele: Stärken

Die Kölner Phonetik definiert 9 unterschiedliche Laute und vergibt ihnen Codes von 0 bis 8.

Sämtliche **Vokale** werden auf "0" abgebildet. Außerdem werden alle "0" im finalen Code gelöscht, bis auf einen potentiellen Vokal am Anfang. Wegen dieser besonderen Behandlung von Vokalen ist die Kölner-Phonetik bei Fehlern in Vokalen besonders invariant.

In [9]:
print(ph.encode("Düsseldorf"))
print(ph.encode("Dassoldurf"))
print(names_by_phonetic["285273"])

285273
285273
{'düsseldorf'}


Die Kodierung der Kölner-Phonetik reduziert außerdem einen Laut, der mehrmals hintereinander vorkommt auf, auf einen.

In [10]:
print(ph.encode("DDDDDüssssselllllddddorf"))

285273


Ähnlichklingende **Konsonanten** haben den selben Laut-Code

In [11]:
print(ph.encode("Düßeldorv"))

285273


## Beispiele: Schwächen

Was die Kölner-Phonetik **nicht** kann, ist gravierende Fehler in den Laut-Codes zu ignorieren. Der Buchstabe "L" erzeugt einen anderen Laut-Code als "D". Somit hat "Lüsseldorf" in der Kölner-Phonetik einen anderen Code.

In [12]:
print(ph.encode("Lüsseldorf"))

585273


Insbesondere das **Hinzufügen von Konsonanten** kann die Kölner Phonetik nicht ignorieren.

In [13]:
print(ph.encode("Trakischken"))
print(ph.encode("Trakirschken"))

274846
2747846


Ebenso das Hinzufügen von ganzen Wörtern verändert den Code.

In [14]:
print(ph.encode("Gr. Trakischken, Goldap"))

472748464521


### Achtung: Unicode, Fremdsprachen, Buchstabe H, unbekannte Laute
Alle folgenden vier Strings besitzen die selbse Codierung in der Kölner Phonetik, was überraschen mag. Das liegt an folgenden vier Gründen:
* H vor einem Vokal wird immer ignoriert
* Für Buchstaben-Kombinationen/Laute aus anderen Sprachen wie "ci" gibt es keine definierte Codierung in der Kölner Phonetik geben, da sie auf die deutsche Sprache optimiert ist. Diese Kombinmationen/Laute werden dann folglich ignoriert.
* Unbekannte (Unicode-)Zeichen wie ż weren ignoriert. Man kann unbekannte Zeichen stets eine der 8 Konsonanten-Laut-Gruppen der Kölner Phonetik zuordnen. Problem: Die Kölner Phonetik ist auf Deutsch optimiert und für Laute fremder Sprache mag es keine passende Gruppe geben.


In [19]:
print(ph.encode("Aachen"))
print(ph.encode("höggen")) ## Buchstabe H am Anfang
print(ph.encode("okocim")) ## Unbekannter Laut in der Kölner Phonetik: "ci"
print(ph.encode("žigoni")) ## Unbekannter Laut/Character in der Kölner Phonetik

046
046
046
046
